In [1]:
import numpy as np

def rosenbrock(x):
    """Rosenbrock function in n dimensions (global minimum at (1,1,...,1))."""
    return sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

def nelder_mead(f, x_start,
                step=0.1,
                no_improve_thr=1e-6,
                no_improv_break=10,
                max_iter=0,
                alpha=1.0, gamma=2.0, rho=0.5, sigma=0.5):
    """
    Nelder-Mead algorithm.
    
    Parameters:
      f (function): The objective function to minimize.
      x_start (numpy array): Initial guess.
      step (float): Initial step size.
      no_improve_thr (float): Threshold for improvement.
      no_improv_break (int): Break after this many iterations with no improvement.
      max_iter (int): Maximum iterations (0 for infinite).
      alpha (float): Reflection coefficient.
      gamma (float): Expansion coefficient.
      rho (float): Contraction coefficient.
      sigma (float): Shrink coefficient.
    
    Returns:
      (best point, best function value, number of iterations)
    """
    dim = len(x_start)
    # initial simplex: x_start plus dim additional points
    simplex = [x_start]
    for i in range(dim):
        x = np.copy(x_start)
        x[i] = x[i] + step
        simplex.append(x)
    simplex = np.array(simplex)
    
    # Evaluate function at vertices
    f_values = np.array([f(x) for x in simplex])
    iterations = 0
    no_improv = 0
    best = min(f_values)
    
    while True:
        # Order vertices by their function values
        indices = np.argsort(f_values)
        simplex = simplex[indices]
        f_values = f_values[indices]
        
        # check for improvement
        if f_values[0] < best - no_improve_thr:
            best = f_values[0]
            no_improv = 0
        else:
            no_improv += 1
        
        # Termination condition
        if max_iter and iterations >= max_iter:
            break
        if no_improv >= no_improv_break:
            break
        
        # Centroid of all points except the worst
        x0 = np.mean(simplex[:-1], axis=0)
        
        # Reflection
        xr = x0 + alpha * (x0 - simplex[-1])
        r_value = f(xr)
        if f_values[0] <= r_value < f_values[-2]:
            simplex[-1] = xr
            f_values[-1] = r_value
        elif r_value < f_values[0]:
            # Expansion
            xe = x0 + gamma * (xr - x0)
            e_value = f(xe)
            if e_value < r_value:
                simplex[-1] = xe
                f_values[-1] = e_value
            else:
                simplex[-1] = xr
                f_values[-1] = r_value
        else:
            # Contraction
            xc = x0 + rho * (simplex[-1] - x0)
            c_value = f(xc)
            if c_value < f_values[-1]:
                simplex[-1] = xc
                f_values[-1] = c_value
            else:
                # Reduction (shrink simplex towards best point)
                for i in range(1, len(simplex)):
                    simplex[i] = simplex[0] + sigma * (simplex[i] - simplex[0])
                    f_values[i] = f(simplex[i])
        iterations += 1

    return simplex[0], f_values[0], iterations

# Example usage:
if __name__ == '__main__':
    # Starting point (for a 2D Rosenbrock function, global minimum is at [1,1])
    x_start = np.array([0.0, 0.0])
    best_point, best_value, iters = nelder_mead(rosenbrock, x_start, max_iter=500)
    print("Best point:", best_point)
    print("Best value:", best_value)
    print("Iterations:", iters)


Best point: [0.99997169 0.99994256]
Best value: 8.686822238627019e-10
Iterations: 73
